In [1]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from collections import Counter
import matplotlib.pyplot as plt
import nltk
import random
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from gensim.models import KeyedVectors
import numpy as np
import re

In [2]:
def cvtCase(s):
    return s.lower()

In [3]:
data = pd.read_csv('/Users/charishmaambati/Desktop/Cherry/Me/Elite/Dataset4E.csv',sep=',', converters={'Question': cvtCase})

In [4]:
def topicQuestion(label):
    return data.loc[(data["Topic"] == label),["Question"]]


In [5]:
def process(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [word for word in tokens]
    return (" ".join(filtered_words))

In [6]:
def preprocessedSampleQuestions(label):
    return [process(item) for item in topicQuestion(label)["Question"]]

In [7]:
def addAllTextSentence(label):
    strData  = " "
    for i in range(len((preprocessedSampleQuestions(label)))):
        strData = strData + (((preprocessedSampleQuestions(label))[i]).lower())+ " "
    return strData

In [8]:
def remTksReSentence(label):
    remRegExp = re.sub(r'[^\w]',' ',addAllTextSentence(label))
    tokens = remRegExp.split()
    return tokens

In [9]:
def preprocessedSampleQuestions(label):
    return [process(item) for item in topicQuestion(label)["Question"]]

In [10]:
labels = list(set(data["Topic"]))

In [11]:
listOfTksReWords = [remTksReSentence(label) for label in labels]

In [12]:
reCommonWords = [process(item) for item in data["Question"] if item not in listOfTksReWords]

In [13]:
epochs = 10
test_accuracy1 = []
val_accuracy1 = []
test_split = [0.1,0.2,0.3]
for epoch in range(epochs):
    print("epoch:", epoch+1)
    test_split = [0.1,0.2,0.3]
    for i in test_split:
        print("test_size", i)
        # Spliting the data into train,test and validation in 60 , 20, 20 respectively
        Q_train, Q_test, T_train, T_test = train_test_split(reCommonWords,data["Topic"] , test_size=i, random_state=2)
        Q_train, Q_val, T_train, T_val  = train_test_split(Q_train, T_train, test_size=0.2, random_state=2)
        ## Applying vectorizer
        vectorizer = CountVectorizer()
        vectorizedTrainQuestions = vectorizer.fit_transform(Q_train)
        encodedTestQuestions = vectorizer.transform(Q_test)
        valTestQuestions = vectorizer.transform(Q_val)
        # Applying multinomialNB
        mnb = MultinomialNB()
        # Fitting the data
        mnb.fit(vectorizedTrainQuestions,T_train)
        # Prediction using test data
        T_prediction = mnb.predict(encodedTestQuestions)
        # Prediction using validation data
        val_prediction = mnb.predict(valTestQuestions)
        if(i == 0.2):
            test_accuracy1.append(accuracy_score(T_prediction,T_test))
        val_accuracy1.append(accuracy_score(val_prediction,T_val))
        print(accuracy_score(T_val, val_prediction))

epoch: 1
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 2
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 3
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 4
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 5
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 6
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 7
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 8
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 9
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429
epoch: 10
test_size 0.1
0.9166666666666666
test_size 0.2
0.90625
test_size 0.3
0.8928571428571429


In [14]:
val_accuracy2 = []
test_accuracy2 = []
test_split = [0.1,0.2,0.3]
epochs = 10
for epoch in range(epochs):
    print("epoch:", epoch+1)
    test_split = [0.1,0.2,0.3]
    for i in test_split:
        print("test_size", i)
        # Spliting the data into train,test and validation in 60 , 20, 20 respectively
        Q_train, Q_test, T_train, T_test = train_test_split(reCommonWords,data["Topic"], test_size=i, random_state=0)
        Q_train, Q_val, T_train, T_val  = train_test_split(Q_train, T_train, test_size=0.2, random_state=0)
        ## Applying vectorizer
        vectorizer = CountVectorizer()
        vectorizedTrainQuestions = vectorizer.fit_transform(Q_train)
        encodedTestQuestions = vectorizer.transform(Q_test)
        valTestQuestions = vectorizer.transform(Q_val)
         # Applying Logistic Regression
        model = LogisticRegression(multi_class='ovr', solver='liblinear')
        # Fitting the data
        model = model.fit(vectorizedTrainQuestions,T_train)
        # Prediction using test data
        predicted = model.predict(encodedTestQuestions)
        # Prediction using validation data
        val_prediction = model.predict(valTestQuestions)
        if(i == 0.2):
            test_accuracy2.append(accuracy_score(predicted,T_test))
        val_accuracy2.append(accuracy_score(val_prediction,T_val))
        print(accuracy_score(T_test,predicted))

epoch: 1
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 2
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 3
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 4
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 5
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 6
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 7
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 8
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 9
test_size 0.1
0.9045454545454545
test_size 0.2
0.9181818181818182
test_size 0.3
0.9287878787878788
epoch: 10
test_size 0.1
0.90

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random
epochs = 10
test_accuracy3 = []
val_accuracy3 = []
train_accuracy3 = []
for epoch in range(epochs):
    print("epoch:", epoch+1)
    test_split = [0.1,0.2,0.3]
    for i in test_split:
        print("test_size", i)
        # Spliting the data into train,test and validation in 60 , 20, 20 respectively
        Q_train, Q_test, T_train, T_test = train_test_split(reCommonWords,data["Topic"] , test_size=i, random_state=0)
        Q_train, Q_val, T_train, T_val  = train_test_split(Q_train, T_train, test_size=0.2, random_state=0)
        ## Applying vectorizer
        vectorizer = TfidfVectorizer(min_df=1)
        vectorizedTrainQuestions = vectorizer.fit_transform(Q_train)
        encodedTestQuestions = vectorizer.transform(Q_test)
        valTestQuestions = vectorizer.transform(Q_val)
        # Applying multinomialNB
        mnb = MultinomialNB()
        # Fitting the data
        mnb.fit(vectorizedTrainQuestions,T_train)
        # Prediction using train data
        Tr_prediction = mnb.predict(vectorizedTrainQuestions)
        # Prediction using test data
        T_prediction = mnb.predict(encodedTestQuestions)
        # Prediction using validation data
        val_prediction = mnb.predict(valTestQuestions)
        train_accuracy3.append(accuracy_score(Tr_prediction,T_train))
        if(i == 0.2):
            test_accuracy3.append(accuracy_score(T_prediction,T_test))
        val_accuracy3.append(accuracy_score(val_prediction,T_val))
        print(accuracy_score(T_test, T_prediction))

epoch: 1
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 2
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 3
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 4
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 5
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 6
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 7
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 8
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 9
test_size 0.1
0.9181818181818182
test_size 0.2
0.8977272727272727
test_size 0.3
0.9272727272727272
epoch: 10
test_size 0.1
0.91

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
test_accuracy4 = []
from sklearn.linear_model import LogisticRegression
import random
test_split = [0.1,0.2,0.3]
epochs = 10
for epoch in range(epochs):
    print("epoch:", epoch+1)
    test_split = [0.1,0.2,0.3]
    for i in test_split:
        print("test_size", i)
        # Spliting the data into train,test and validation in 60 , 20, 20 respectively
        Q_train, Q_test, T_train, T_test = train_test_split(reCommonWords,data["Topic"] , test_size=i, random_state=0)
        Q_train, Q_val, T_train, T_val  = train_test_split(Q_train, T_train, test_size=0.2, random_state=0)
        ## Applying vectorizer
        vectorizer = TfidfVectorizer(min_df=1,stop_words='english')
        vectorizedTrainQuestions = vectorizer.fit_transform(Q_train)
        encodedTestQuestions = vectorizer.transform(Q_test)
        # Applying Logistic Regression
        model = LogisticRegression()
        # Fitting the data
        model = model.fit(vectorizedTrainQuestions,T_train)
        # Prediction using test data
        predicted = model.predict(encodedTestQuestions)
        if(i == 0.2):
            test_accuracy4.append(accuracy_score(predicted,T_test))
        print(accuracy_score(T_test, predicted))
print(test_accuracy4)

epoch: 1
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 2
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 3
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 4
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 5
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 6
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 7
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 8
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 9
test_size 0.1
0.8954545454545455
test_size 0.2
0.9136363636363637
test_size 0.3
0.9287878787878788
epoch: 10
test_size 0.1
0.89

In [17]:
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [18]:
def convertWV(para):
    output = []   
    for line in para:
        line = line.split(" ")
        inn = []
        for i in range(len(line)):
            try:
                x = model[line[i].lower()]
            except:
                x = np.zeros(300)
            inn.append(x)
        output.append(inn)
    w2v = [sum(element) for element in output]
    return w2v

In [19]:
epochs = 10
accuracy = []
testSplit = [0.1,0.2,0.3]
for epoch in range(epochs):
    print("epochs:",epoch + 1)
    for size in testSplit:
        print("testSize:",size)
        questionTrain, questionTest, labelTrain, labelTest =train_test_split(reCommonWords,data["Topic"] , test_size = size)
        questionTrain, questionVal, labelTrain, labelVal =train_test_split(questionTrain, labelTrain, test_size = 0.2)
        lR = LogisticRegression()
        lR = lR.fit (convertWV(questionTrain),labelTrain)
        probability = lR.predict_proba(convertWV(questionTest))
        predicted = lR.predict(convertWV(questionTest))
        if(size == 0.2):
            accuracy.append(accuracy_score(labelTest,predicted))
        print(accuracy_score(labelTest,predicted))
print(accuracy)

epochs: 1
testSize: 0.1
0.9545454545454546
testSize: 0.2
0.9363636363636364
testSize: 0.3
0.9090909090909091
epochs: 2
testSize: 0.1
0.9318181818181818
testSize: 0.2
0.9545454545454546
testSize: 0.3
0.9257575757575758
epochs: 3
testSize: 0.1
0.9409090909090909
testSize: 0.2
0.9318181818181818
testSize: 0.3
0.9257575757575758
epochs: 4
testSize: 0.1
0.9363636363636364
testSize: 0.2
0.925
testSize: 0.3
0.9151515151515152
epochs: 5
testSize: 0.1
0.9227272727272727
testSize: 0.2
0.9295454545454546
testSize: 0.3
0.9318181818181818
epochs: 6
testSize: 0.1
0.9
testSize: 0.2
0.9227272727272727
testSize: 0.3
0.9242424242424242
epochs: 7
testSize: 0.1
0.9227272727272727
testSize: 0.2
0.9454545454545454
testSize: 0.3
0.9272727272727272
epochs: 8
testSize: 0.1
0.9
testSize: 0.2
0.9318181818181818
testSize: 0.3
0.9272727272727272
epochs: 9
testSize: 0.1
0.9318181818181818
testSize: 0.2
0.925
testSize: 0.3
0.9303030303030303
epochs: 10
testSize: 0.1
0.9227272727272727
testSize: 0.2
0.918181818181818